In [16]:
# unsuccessfull attempt at exercise 2

import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F

In [9]:
def generate_addition_problem():
    a = random.randint(0, 999)
    b = random.randint(0, 999)
    c = a + b
    problem = f'{a}+{b}='
    solution = str(c)[::-1]
    # we reverse the solution (e.g., "579" becomes "975") to simulate right-to-left addition as typical in manual addition
    return problem, solution

def create_dataset(size):
    data = []
    for _ in range(size):
        problem, solution = generate_addition_problem()
        data.append((problem, solution))
    return data

text = create_dataset(10000)

In [14]:
vocab = '0123456789+='
vocab_size = len(vocab)

stoi = {ch:i for i, ch in enumerate(vocab)}
itos = {i:ch for i, ch in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]  # encoder: takes a string, returns a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: takes a list of integers, returns a string

In [15]:
def generate_batch(batch_size):
    problems, solutions = [], []
    for _ in range(batch_size):
        a = random.randint(0, 999)
        b = random.randint(0, 999)
        c = a + b
        problem = f'{a}+{b}='
        solution = str(c)[::-1]
        problems.append(problem)
        solutions.append(solution)
    return problems, solutions        

In [22]:
class GPTAddition(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 100, d_model))  # Max sequence length of 100
        self.transformer = nn.Transformer(d_model, nhead, num_layers, num_layers, d_model*4, batch_first=True)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer(x, x)
        return self.fc_out(x)

In [34]:
# batch_size = 64
# optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
# max_iters = 1000

model = GPTAddition(vocab_size, d_model=256, nhead=8, num_layers=6)
optimizer = torch.optim.Adam(model.parameters())
train(model, optimizer, max_iters=1000, batch_size=64)

In [32]:
def train(model, optimizer, max_iters, batch_size):
    for iter in range(max_iters):
        total_loss = 0
        for _ in range(100):  # 100 batches per epoch
            problems, solutions = generate_batch(batch_size)
 
            x = torch.tensor([encode(p) for p in problems], dtype=torch.long)
            y = torch.tensor([encode(s) for s in solutions], dtype=torch.long)
    
            # create target mask
            target_mask = torch.ones_like(x) * -1  # -1 is used as a placeholder to indicate positions that should be ignored by the loss function during training
            for i, problem in enumerate(problems):
                target_mask[i, len(problem):] = y[i, : len(y[i])]
    
            y_pred = model(x)
            loss = F.cross_entropy(y_pred.view(-1, tokenizer.vocab_size), target_mask.view(-1), ignore_index=-1)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
        print(f"Epoch {iter+1}/{max_iters}, Loss: {total_loss/100}")
    
    torch.save(model.state_dict(), 'gpt_addition_model.pth')